In [ ]:
import numpy as np
import torch
import torch.nn as nn
from metatensor import TensorMap, Labels, TensorBlock

In [ ]:
class model(nn.Module):
    def __init__(self,
                 features,
                 device=None,
                 **kwargs):

        super().__init__()
        self.features = features 
        self.device = 'cpu'
        self.bias = False
        self.model = nn.Sequential(
                nn.Linear(self.features[0].values.shape[-1],100, bias=self.bias),
                nn.Linear(100,50, bias=self.bias),
                nn.Linear(50,100, bias=self.bias),
                nn.Linear(100,self.features[0].values.shape[-1],bias=self.bias)
        ).to(self.device)

    def forward(self, features = None):
        pred_blocks = []
        if features is None:
            features = self.features
        keys = features.keys
        for k, feat in features.items():
            pred = self.model(feat.values)
            pred_blocks.append(
                    TensorBlock(
                        values = pred,
                        samples = feat.samples,
                        components = feat.components,
                        properties = feat.properties,
                    )
                )
            pred = None
        
        return TensorMap(keys, pred_blocks)

    def forward_nomts(self, features = None):
        pred_blocks = []
        if features is None:
            features = self.features
        keys = features.keys
        for k, feat in features.items():
            pred = self.model(feat.values)
            pred_blocks.append(pred)
        return pred_blocks

In [ ]:
keys = Labels(['order_nu','inversion_sigma', 'spherical_harmonics_l', 'species_center', 'species_neighbor', 'block_type'], np.array([[0,0,0,0,0,i] for i in range(10)]))

n_s = 10000
n_p = 600
block = [TensorBlock(values = torch.randn(n_s,1,n_p), 
                   samples= Labels(['s'], values = np.arange(37, 37 + n_s).reshape(n_s,1)),
                   components = [Labels(['c'], values=np.array([[0]]))],
                   properties= Labels(['p'], values = np.arange(73, 73 + n_p).reshape(n_p,1))
) for i in range(10)]
dfeat = TensorMap(keys, block)

In [ ]:
dmodel = model(dfeat)

This is fine (the memory stays ~constant)

In [ ]:
for epoch in range(1000):
    print(epoch, end = ' ')
    dmodel.forward_nomts()

This is not fine (the memory increases at every few epochs)

In [ ]:
for epoch in range(1000):
    print(epoch, end = ' ')
    dmodel.forward()